In [1]:
#import des fichiers et load des ressources
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('2016_Building_Energy_Benchmarking.csv')
pd.options.display.max_columns = data.shape[1]

In [2]:
data.sample(1)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
644,19445,2016,NonResidential,Low-Rise Multifamily,Pacific Pointe,2108 N Pacific St.,Seattle,WA,98103.0,0354000254,4,LAKE UNION,47.64859,-122.33311,2000,1.0,3,93079,36093,56986,Multifamily Housing,Multifamily Housing,91064.0,NaN,NaN,NaN,NaN,NaN,32.0,21.9,23.0,63.099998,65.099998,1997181.5,2090011.625,0.0,511195.5938,1744199.0,2529.819824,252982.0,False,NaN,Compliant,NaN,25.6,0.28


In [3]:
#variables cohérentes
cols = ['OSEBuildingID', 'DataYear', 'BuildingType',
       'PropertyName','Neighborhood','YearBuilt','NumberofBuildings','NumberofFloors',
        'PropertyGFATotal','PropertyGFAParking','PropertyGFABuilding(s)',
       'LargestPropertyUseType', 'ENERGYSTARScore',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
        'Electricity(kBtu)','NaturalGas(kBtu)','TotalGHGEmissions']
data = data[cols]

In [4]:
data['BuildingType'].value_counts()

NonResidential          1460
Multifamily LR (1-4)    1018
Multifamily MR (5-9)     580
Multifamily HR (10+)     110
SPS-District K-12         98
Nonresidential COS        85
Campus                    24
Nonresidential WA          1
Name: BuildingType, dtype: int64

In [5]:
data_length_before = data.shape[0]
data['BuildingType'].replace({'Nonresidential COS': 'NonResidential',
                              'Nonresidential WA': 'NonResidential'}, inplace=True)
data = data.loc[data['BuildingType'] == 'NonResidential']

print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  1830


In [6]:
#valeurs manquantes
data.isna().sum()

OSEBuildingID               0
DataYear                    0
BuildingType                0
PropertyName                0
Neighborhood                0
YearBuilt                   0
NumberofBuildings           2
NumberofFloors              0
PropertyGFATotal            0
PropertyGFAParking          0
PropertyGFABuilding(s)      0
LargestPropertyUseType      6
ENERGYSTARScore           540
SiteEnergyUseWN(kBtu)       3
SteamUse(kBtu)              2
Electricity(kBtu)           2
NaturalGas(kBtu)            2
TotalGHGEmissions           2
dtype: int64

In [7]:
#drop toutes lignes qui ont des valeurs nulles sauf EnergyScore
data_length_before = data.shape[0]
cols = ['LargestPropertyUseType', 'SiteEnergyUseWN(kBtu)', 'TotalGHGEmissions', 'NaturalGas(kBtu)', 'Electricity(kBtu)', 'SteamUse(kBtu)', 'NumberofBuildings']
data.dropna(subset=cols, inplace=True)
data.isna().sum()
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  7


In [8]:
data.describe()

,OSEBuildingID,DataYear,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ENERGYSTARScore,SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions
count,1539.000000,1539.0,1539.000000,1539.000000,1539.000000,1.539000e+03,1539.000000,1.539000e+03,1003.000000,1.539000e+03,1.539000e+03,1.539000e+03,1.539000e+03,1539.000000
mean,16609.779727,2016.0,1961.307342,1.035737,4.283951,1.120106e+05,13829.220273,9.818137e+04,63.790628,7.761522e+06,4.967574e+05,5.435044e+06,1.672264e+06,165.047498
std,13858.481096,0.0,32.888052,0.569622,6.796834,1.918960e+05,43730.728115,1.685664e+05,28.923639,1.884349e+07,5.314667e+06,1.305847e+07,5.513280e+06,571.657895
min,1.000000,2016.0,1900.000000,0.000000,0.000000,1.128500e+04,0.000000,3.636000e+03,1.000000,0.000000e+00,0.000000e+00,-1.154170e+05,0.000000e+00,-0.800000
25%,609.500000,2016.0,1929.000000,1.000000,1.000000,2.872800e+04,0.000000,2.768500e+04,44.000000,1.265929e+06,0.000000e+00,7.075250e+05,0.000000e+00,19.335000
50%,21213.000000,2016.0,1965.000000,1.000000,2.000000,4.721500e+04,0.000000,4.508200e+04,71.000000,2.739122e+06,0.000000e+00,1.633105e+06,4.313920e+05,48.350000
75%,24629.500000,2016.0,1988.000000,1.000000,4.000000,1.028880e+05,0.000000,9.290600e+04,89.000000,7.227405e+06,0.000000e+00,5.046944e+06,1.409886e+06,136.735000
max,50226.000000,2016.0,2015.000000,9.000000,99.000000,1.952220e+06,512608.000000,1.765970e+06,100.000000,2.966717e+08,1.349435e+08,2.745325e+08,1.381912e+08,12307.160000


In [9]:
data_length_before = data.shape[0]
cols = data.select_dtypes(include=[np.number]).columns
data = data[~(data[cols] < 0).any(axis=1)].copy()
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  1


In [10]:
data_length_before = data.shape[0]
difference = abs(data['PropertyGFATotal'] - (data['PropertyGFABuilding(s)'] + data['PropertyGFAParking']))
data = data[difference == 0]
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  0


In [11]:
#création d'une colonne ancienneté à partir des variable DataYear et YearBuilt
data['Ancienneté'] = data['DataYear'] - data['YearBuilt']
data = data.drop('YearBuilt', axis=1)


In [12]:
numeric_cols = ['NumberofFloors','NumberofBuildings','PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'ENERGYSTARScore', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions', 'Ancienneté']
df_numeric = data[numeric_cols]
scaler = StandardScaler()
df_numeric_scaled = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)
data[numeric_cols] = df_numeric_scaled

In [13]:
#energy non nulle
data_energy_star_filtered = data[data['ENERGYSTARScore'].notna()]
data = data[data['ENERGYSTARScore'].notna()]

In [14]:
data = pd.concat([data, pd.get_dummies(data['Neighborhood'], prefix='Neighborhood')], axis=1)
data = pd.concat([data, pd.get_dummies(data['LargestPropertyUseType'], prefix='LargestPropertyUseType')], axis=1)
dummy_cols = data.filter(regex='^(LargestPropertyUseType_|Neighborhood_)')
data = data.loc[:, (data != 0).any(axis=0) | (dummy_cols != 0).any(axis=0)]


In [15]:
data.to_csv('cleaned_data.csv', index=False)

In [16]:
data.shape[0]

621

In [17]:
data

,OSEBuildingID,DataYear,BuildingType,PropertyName,Neighborhood,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,ENERGYSTARScore,SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,Ancienneté,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DOWNTOWN,Neighborhood_Delridge,...,LargestPropertyUseType_Other - Education,LargestPropertyUseType_Other - Entertainment/Public Assembly,LargestPropertyUseType_Other - Lodging/Residential,LargestPropertyUseType_Other - Mall,LargestPropertyUseType_Other - Public Services,LargestPropertyUseType_Other - Recreation,LargestPropertyUseType_Other - Restaurant/Bar,LargestPropertyUseType_Other - Services,LargestPropertyUseType_Parking,LargestPropertyUseType_Refrigerated Warehouse,"LargestPropertyUseType_Repair Services (Vehicle, Shoe, Locksmith, etc)",LargestPropertyUseType_Residence Hall/Dormitory,LargestPropertyUseType_Residential Care Facility,LargestPropertyUseType_Restaurant,LargestPropertyUseType_Retail Store,LargestPropertyUseType_Self-Storage Facility,LargestPropertyUseType_Senior Care Community,LargestPropertyUseType_Social/Meeting Hall,LargestPropertyUseType_Strip Mall,LargestPropertyUseType_Supermarket/Grocery Store,LargestPropertyUseType_Urgent Care/Clinic/Other Outpatient,LargestPropertyUseType_Wholesale Club/Supercenter,LargestPropertyUseType_Worship Facility
0,1,2016,NonResidential,Mayflower park hotel,DOWNTOWN,-0.06278,1.135429,-0.123069,-0.316452,-0.058005,Hotel,-0.129909,-0.016426,0.283518,-0.114310,-0.071992,0.148388,1.042972,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,2016,NonResidential,Paramount Hotel,DOWNTOWN,-0.06278,0.988299,-0.044211,0.028031,-0.057601,Hotel,-0.095308,0.047662,-0.093530,-0.168161,0.629722,0.228648,-1.056743,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2016,NonResidential,5673-The Westin Seattle,DOWNTOWN,-0.06278,5.402210,4.398669,4.182088,3.922471,Hotel,-0.718124,3.511777,3.964413,3.376398,-0.032568,3.365960,-0.235115,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,2016,NonResidential,HOTEL MAX,DOWNTOWN,-0.06278,0.841169,-0.264368,-0.316452,-0.218859,Hotel,-0.268312,-0.043498,0.323138,-0.204456,0.025006,0.212151,1.073402,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,2016,NonResidential,WARWICK SEATTLE HOTEL (ID8),DOWNTOWN,-0.06278,2.018212,0.331077,1.101362,0.091175,Hotel,0.389105,0.365667,-0.093530,-0.005364,1.293397,0.594523,-0.569852,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,21945,2016,NonResidential,First Hill Plaza Retail Building,EAST,-0.06278,-0.041614,-0.258839,0.208322,-0.348704,Parking,-0.060707,-0.226573,-0.093530,-0.148681,-0.303522,-0.246265,-1.574064,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1509,22142,2016,NonResidential,Safeway 0219 - Othello St,GREATER DUWAMISH,-0.06278,1.429690,0.169703,-0.316452,0.275285,Supermarket/Grocery Store,0.873517,-0.117736,-0.093530,-0.012977,-0.257112,-0.200887,-1.634925,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1513,22165,2016,NonResidential,(ID22165) MICHIGAN STREET BUILDINGS,GREATER DUWAMISH,-0.06278,1.135429,1.935210,3.484386,1.299090,Non-Refrigerated Warehouse,1.184925,0.352556,-0.093530,0.482042,0.107429,0.064664,-1.634925,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1514,22166,2016,NonResidential,Georgetown Center Bldg A,GREATER DUWAMISH,-0.06278,1.282560,2.212974,4.203606,1.428710,Retail Store,1.184925,0.332798,-0.093530,0.343637,0.297459,0.139956,-1.604494,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
